## **Workshop on how to Bridge the gap between Neuroscience & Computational Intelligence for BCI**
## See more detail about workshop here: https://thinknew.github.io/BCIWorkshop/
This notebook aims to demonstrate and teach the following content:



# Setting Up the Environment

The prerequisite packages to this tutorial are:


*   [**MNE**](https://mne.tools/stable/install/mne_python.html#installing-mne-python-and-its-dependencies): EEG Data Package
*   [**NumPy**](https://www.scipy.org/install.html): Numerical Computing (SciPy Download Page)
*   [**SciPy**](https://www.scipy.org/install.html): Scientific Computing
*   [**MatPlotLib**](https://matplotlib.org/users/installing.html): General Purpose Machine Learning Library
*   [**Scikit-Learn**](https://scikit-learn.org/stable/install.html): General Purpose Machine Learning Library
*   [**Tensorflow 2.0**](https://www.tensorflow.org/guide/effective_tf2/): Deep Learning Library from Google Brain

All packages but MNE are available through Google Colab and thus we only need to install MNE.

In [ ]:
a = 5

In [ ]:
# Run these from the console if following along locally
!pip install mne
!pip install tensorboardX
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 75.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 23.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.3 MB/s eta 0:00:00


For the purposes of this workshop, we will be turning off warnings.

In [ ]:
# You'll want to comment this out if you plan on modifying this code, to get valuable feedback
import warnings
warnings.filterwarnings('ignore')

Import all of our required modules. This includes various submodules that we'll need including the RobustScaler object from scikit.

In [ ]:
from collections import OrderedDict
from pylab import rcParams
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.cm import get_cmap
import numpy as np
import mne
from scipy.io import loadmat
import sklearn
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
import h5py
import json

from einops import rearrange

from google.colab import drive, files

## Reproducibility Matters

In [ ]:
# Set seed to reproduce behaviour
tf.random.set_seed(100)

# Data Analysis

## Load the EEG data
Here we will load both the raw and preprocessed data.

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
# Define the names of our files
# TODO: Are we using json? .mat?
raw_file = './OutputDataAppJson.json'
processed_file = './OutputDataAppJson.json'

Extract our raw data. Plot the data to verify that it was loaded correctly.

In [ ]:
with open(raw_file, 'r') as f:
  raw_p300_json = json.load(f)
# P300_file = loadmat(raw_file)
print(list(raw_p300_json.keys()))
#raw_p300_json['Data']

In [ ]:
# TODO: Not sure how to make this nice
channels = raw_p300_json['ChannelLabels']
raw_data = np.array(raw_p300_json['Data'][:4])
# plot
rcParams['figure.figsize'] = 15, 7
scaler = 3e-5
c, x, y = raw_data.shape
a = raw_data.reshape((x, c, y))
x = np.arange(x)
c_color = ['tab:blue', 'tab:red', 'tab:green', 'tab:orange']
for i, c_data in enumerate(raw_data):
  plt.subplot(c//2,2,i+1)
  plt.title(channels[i])
  for j, data in enumerate(c_data.T):
    data += j*scaler
    plt.plot(data, color=c_color[i])

Extract our processed data. Again, plot the data to verify that it was loaded correctly.

In [ ]:
with open(processed_file, 'r') as f:
  proc_json = json.load(f)
# P300_file = loadmat(processed_file)
processed_file
proc_json.keys()

In [ ]:
with open('fakedata.mat', 'rb') as f:
  fakedata = loadmat(f)
print(fakedata.keys())

with open('Allen_WithScenario_P300_2.mat', 'rb') as f:
  scenario = loadmat(f)
print(scenario.keys())

In [ ]:
dset_choice = 'scenario'
if dset_choice == 'fakedata':
  selected_data = fakedata
else:
  selected_data = scenario

In [ ]:
try:
  data = selected_data['data']
  labels = selected_data['labels']
except:
  data = selected_data['exportdata']
  labels = selected_data['exportlabel']

In [ ]:
data.shape

Set the plotting parameters "scale, num channels"

In [ ]:
# plot
#channels = proc_json['ChannelLabels']
#proc_data = np.array(proc_json['Data'])
proc_data = np.array(data)
fs = proc_json['srate']
# plot
rcParams['figure.figsize'] = 15, 7
scaler = 0.0001
c, x, y = proc_data.shape
a = proc_data.reshape((x, c, y))
x = np.arange(x)
c_color = get_cmap('tab10').colors

n_cols = 2
# n_rows = np.ceil(c/n_cols).astype(int)
n_rows = 2
fig, ax = plt.subplots(n_rows, n_cols, sharex=True)
ax = ax.flatten()

# lets print only the first 4 channels and 10 epochs
for i, c_data in enumerate(proc_data[:4]):
  ax[i].set_title(channels[i])
  for j, plot_data in enumerate(c_data.T):
    plot_data += j*scaler
    ax[i].plot(plot_data, color=c_color[i])
    if j > 9: break

We need to scale and split our data to feed into our model for training.

In [ ]:
P300_data = proc_data
# select the 5-th channel as the training data
# P300_data_c = P300_data[3, :]
P300_data_c = P300_data


In [ ]:
# TODO not implemented
# label_shape = P300_data_c.shape[-1]
# P300_label = np.zeros(label_shape, dtype=bool)
# rand_pos = np.random.randint(0, len(P300_label), P300_label.shape)
# P300_label[rand_pos] = True
# print(P300_label)
if dset_choice == 'fakedata':
  P300_label = labels
else:
  P300_label = labels.T
c, _, e = P300_data_c.shape
if e > c:
  P300_data_c = np.swapaxes(P300_data_c, 0, -1)
print(P300_data_c.shape)
print(P300_label.shape)

from sklearn.model_selection import train_test_split
P300_data_c_train, P300_data_c_test, P300_label_c_train, P300_label_c_test = train_test_split(P300_data_c, P300_label, test_size=0.2, random_state=123)

print(P300_data_c_train.shape)
print(P300_data_c_test.shape)
print(P300_label_c_train.shape)
print(P300_label_c_test.shape)

In [ ]:
def noise_augment_channel(data, scale=0.5, axis=-1):
  for c in range(data.shape[axis]):
    ch = data[:,c]
    mu = ch.mean()
    sd = ch.std()
    data[:,c] = ch + np.random.normal(scale*mu, scale=scale*sd, size=ch.shape)
  return data

def noise_augment_bulk(data, scale=0.5, axis=-1):
  mu = data.mean()
  sd = data.std()
  data = data + scale*np.random.normal(mu, scale=sd, size=data.shape[0]).reshape(-1,1)
  return data

def bias_augment(data, scale=0.5, axis=-1):
    bias_range = scale*(data.max() - data.min())
    aug = np.random.random(data.shape[axis])*bias_range
    print(aug.shape)
    return data+aug

In [ ]:
# Separate positive examples
nbins = len(P300_label_c_train)
inds = np.arange(nbins)
pos_inds = inds[P300_label_c_train.astype(bool).flatten()]
data_in = P300_data_c_train[pos_inds]
lbl_in = P300_label_c_train[pos_inds]
diff = nbins - len(pos_inds)
print(diff)

In [ ]:
test = noise_augment_bulk(data_in[0].copy())
plt.plot(data_in[0,:,0])
plt.plot(test[:,0])

In [ ]:
# data_in = P300_data_c_train[0, :, c].reshape(-1,1)
aug_data = []
for n in range(diff):
  select_ind = np.random.randint(len(pos_inds))
  select_data = data_in[select_ind].copy()
  data_gauss = noise_augment_bulk(select_data)
  aug_data.append(data_gauss)

aug_data = np.array(aug_data)
print(aug_data.shape)

In [ ]:
nfig = 3
c = 0
for i in range(nfig):
  plt.subplot(nfig,1,i+1)
  plt.plot(data_in[i, :, c])
  plt.plot(aug_data[i, :, c])

In [ ]:
new_lbls = np.ones((len(aug_data), 1))
x_aug_dset = np.concatenate((P300_data_c_train, aug_data), axis=0)
y_aug_dset = np.concatenate((P300_label_c_train, new_lbls), axis=0)

print(x_aug_dset.shape)
print(y_aug_dset.shape)

In [ ]:
## NORMALIZING
def normalize_dset(data):
  n_channels = data.shape[0]
  for c in range(n_channels):
    data_to_scale = data[c]
    scaler = MinMaxScaler()
    data[c] = scaler.fit_transform(data_to_scale)
  return data

In [ ]:
P300_data_c_train = normalize_dset(x_aug_dset)
P300_data_c_test = normalize_dset(P300_data_c_test)

P300_label_c_train = y_aug_dset

# Neural Network

Initialize parameters including EEG related Metadata.


*   **eeg_sample_count**:  Number of samples we're training our network with
*   **learning_rate**:     How fast the network tends to change its weights
*   **eeg_sample_length**: Number of datapoints per sample
*   **number_of_classes**: Number of output classes (a scalar value that represents probability of input belonging to p300)
*   **hidden1**: Number of neurons in the first hidden layer
*   **hidden2**: Number of neurons in the second hidden layer
*   **hidden3**: Number of neurons in the third hidden layer
*   **output**: Number of neurons in the output layer

In [ ]:
# Initialize parameters
eeg_sample_count = 240 # How many samples are we training
epochs = 10 # number of times we loop through our training data
learning_rate = 1e-3 # How hard the network will correct its mistakes while learning
# loss = 'mse' # Metric for our optimizer to minimise
loss = keras.losses.BinaryFocalCrossentropy()
batch_size = 32 # Number of samples in our batch
eeg_sample_length = 226 # Number of eeg data points per sample
number_of_classes = 1 # We want to answer the "is this a P300?" question
hidden1 = 500 # Number of neurons in our first hidden layer
hidden2 = 1000 # Number of neurons in our second hidden layer
hidden3 = 100 # Number of neurons in our third hidden layer
penultimate = 10 # Number of neurons in our output layer
metrics = ['binary_accuracy'] # Metrics for us to monitor during training and tuning

We use our hyperparameters to define our neural network model.

In [ ]:
## Define the network
P300_model = keras.Sequential()

# Input Layer (Size 226 -> 500)
P300_model.add(layers.Dense(hidden1, activation="relu", name="Input_Layer"))

# Hidden Layer (Size 500 -> 1000)
P300_model.add(layers.Dense(hidden2, activation="relu", name="Hidden_Layer1"))

# Hidden Layer (Size 1000 -> 100)
P300_model.add(layers.Dense(hidden3, activation="relu", name="Hidden_Layer2"))

# Hidden Layer (Size 100 -> 10)
P300_model.add(layers.Dense(penultimate, activation="relu", name="Hidden_Layer3"))

# Handle multi-channel inputs
P300_model.add(layers.Flatten())

# Output Layer (Size 10 -> 1)
P300_model.add(layers.Dense(number_of_classes, activation="sigmoid", name="Output_Layer"))

# Define a learning function, needs to be reinitialized every load
optimizer = keras.optimizers.Adam(0.001)
# After the model is created, we then config the model with losses and metrics
P300_model.compile(optimizer=optimizer,
             loss=loss,
             metrics=metrics)

In [ ]:
list0 = [1]*5
list1 = [2]*3

for x,y in zip(list0, list1): print(x+y)


In [ ]:
print(len(P300_data_c_train))
print(len(P300_label_c_train))
nbatches = len(P300_data_c_train)//batch_size

x_ds = tf.data.Dataset.from_tensor_slices(P300_data_c_train)
y_ds = tf.data.Dataset.from_tensor_slices(P300_label_c_train)

ds = tf.data.Dataset.zip((x_ds, y_ds))\
  .batch(batch_size)\
  .shuffle(1000)

train_size = int(nbatches*0.7)
train_ds = ds.take(train_size)
val_ds = ds.skip(train_size)

for x,y in ds.take(1).as_numpy_iterator(): print(x.shape)
print(train_ds.cardinality())
print(val_ds.cardinality())

Now that we have our model defined, we can train and test on our dataset.

In [ ]:
# Make sure we're starting from untrained every time
#tutorial_model = keras.models.load_model('/home/tutorial_model_default_state')
#tf.compat.v1.initializers.global_variables()
mse_list = P300_model.fit(ds, epochs=epochs, verbose=1)

# Define a learning function, needs to be reinitialized every load
# optimizer = torch.optim.Adam(tutorial_model.parameters(), lr = learning_rate)

# Use our training procedure with the sample data
print("Below is the loss graph for dataset training session")
loss_data = mse_list.history['loss']
# Plot a nice loss graph at the end of training
rcParams['figure.figsize'] = 10, 5
plt.title("Loss vs Iterations")
plt.plot(list(range(0, len(loss_data))), loss_data)
plt.show()

## Classification Performance

In [ ]:
# Classify our positive test dataset and print the results
classification_test = np.round(P300_model.predict(P300_data_c_test))
print("classification max: ", classification_test.max())
print(classification_test.shape)
from sklearn.metrics import classification_report
print(classification_report(P300_label_c_test, classification_test))

# Review
How did your model do? How did your model do with you data after signal processing? What hyperparameters significantly affected the performance?